In [1]:
%reset -f
import os, pandas as pd, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt, tensorflow as tf
from matplotlib.ticker import FormatStrFormatter
mpl.rcParams['figure.dpi'] = 100

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
USE_GPU = False

if not USE_GPU:
    try:
        # Disable all GPUSs
        tf.config.set_visible_devices([], 'GPU')
        visible_devices = tf.config.get_visible_devices()
        for device in visible_devices:
            assert device.device_type != 'GPU'
    except:
        print("failed to disable gpu")
        # Invalid device or cannot modify virtual devices once initialized.
        pass
print(tf.config.get_visible_devices())
tf.random.set_seed(42)

from modules.param_wgan_gp import load_model
mname, epoch = "CM_small_dLNkiHU", 69
print(mname, epoch)
gan = load_model(f"models/{mname}/model_params.json", 
                    f"models/{mname}/epoch{epoch}/generator_weights.h5", 
                    f"models/{mname}/epoch{epoch}/discriminator_weights.h5")
col_ns = list(gan.pd['feat_names'])
print(col_ns)

mceg_df = pd.DataFrame(dict(zip(list(gan.pd['feat_names']), np.load("data/raw/X.npy").T)))#[:10000000]
mceg_df.phih = mceg_df.phih - 360*(mceg_df.phih>180)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
CM_small_dLNdLLEdKR 69


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'models/CM_small_dLNdLLEdKR/epoch69/generator_weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# TOPEG interactive: real 362m6.368s, user 361m51.997s, sys 0m3.475s
# TOPEG batch nodes: 36080.01user 74.13system 10:02:37elapsed 99%CPU (0avgtext+0avgdata 398000maxresident)k 3120inputs+15528outputs (3major+567529minor)pagefaults 0swaps

# CM_small_dLNdKR CPU: 278 ms ± 3.61 ms
# %timeit sclr.inverse_transform(wgan.gen(tf.random.normal(shape=(100000, param_dict['latent_dim'])), training=False))
# %timeit gan.generate(n_samples=len(mceg_df), batch_size=1000000, verbose=False, pandas_df=False, gpu=USE_GPU)
gan_df = gan.generate(n_samples=len(mceg_df), batch_size=1000000, verbose=False, pandas_df=True, gpu=USE_GPU)

In [ ]:
def sideBySide2D(x, y, xyr, bins=200, xl=None, yl=None, title=None, chi2=False, scilabs=False):
    f, axs = plt.subplots(1,2, figsize=(20,10))
    
    hhm, xe, ye = np.histogram2d(mceg_df[x], mceg_df[y], bins=bins, range=xyr)
    hhg, xe, ye = np.histogram2d( gan_df[x],  gan_df[y], bins=bins, range=xyr)
    vmin, vmax = np.min([hhm, hhg]), np.max([hhm, hhg])
    norm = mpl.colors.LogNorm(vmin=1, vmax=vmax)

    axs[0].hist2d(mceg_df[x], mceg_df[y], bins=[xe,ye], range=xyr, cmap='cividis', norm=norm)
    axs[1].hist2d( gan_df[x],  gan_df[y], bins=[xe,ye], range=xyr, cmap='cividis', norm=norm)

    for i, ax in enumerate(axs):
        ax.set_title(f"{'MCEG' if i else 'GAN'}{' - '+title if title else ''}", fontsize=26)
        ax.set_xlabel(xl if xl else x, fontsize=22)
        if scilabs:
            ax.ticklabel_format(axis=scilabs,style='sci', scilimits=(0,0), useMathText=True)
        ax.tick_params(axis='both', which='major', labelsize=18)
        ax.grid()
        if not i:
            ax.set_ylabel(yl if yl else y, fontsize=22)
    
    return f, axs

def contourOn2D(x, y, xyr, bins=200, levels=15, xl=None, yl=None, title=None, chi2=False, scilabs=False):
    f, ax = plt.subplots(1,1, figsize=(10,10))

    hhm, xe, ye = np.histogram2d(mceg_df[x], mceg_df[y], bins=bins, range=xyr)
    hhg, xe, ye = np.histogram2d( gan_df[x],  gan_df[y], bins=bins, range=xyr)
    vmin, vmax = np.min([hhm, hhg]), np.max([hhm, hhg])
    norm = mpl.colors.LogNorm(vmin=1, vmax=vmax)
    levels = np.logspace(0, np.log10(vmax), levels+1)

    extent = [xe.min(), xe.max(), ye.min(), ye.max()]
    csm = ax.contour( hhm.T, extent=extent, linewidths=2, colors='k', levels=levels[2:])
    csg = ax.contourf(hhg.T, extent=extent, cmap='jet', norm=norm, levels=levels)

    chi2 = np.mean((hhm-hhg)**2/np.fmax(1,hhm))

    ax.set_title(r'GAN with MCEG Contours, $\chi^2$ = '+f'{chi2:.2f}', fontsize=18)
    ax.set_xlabel(xl, fontsize=16)
    ax.set_ylabel(yl, fontsize=16)
    if scilabs:
        ax.ticklabel_format(axis=scilabs,style='sci', scilimits=(0,0), useMathText=True)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.grid()

    return f, ax


In [ ]:
bins=1000
xlabs = [r'$Q^2$ [GeV$^2$]', r'$x_B$', r'$y$', r'$-t$ [GeV$^2$]', r'$\varphi_{HADRON}$']
for log in [False, True]:
    f, axs = plt.subplots(2,3, figsize=(20,10))
    for cn, ax, xl in zip(col_ns, axs.flatten()[:-1], xlabs):
        bc = np.linspace(np.min(np.r_[mceg_df[cn], gan_df[cn]]), np.max(np.r_[mceg_df[cn],gan_df[cn]]), bins+1)
        hhm, _, _ = ax.hist(mceg_df[cn], bins=bins, histtype='step', linewidth=2)
        hhg, _, _ = ax.hist(gan_df[cn], bins=bins, histtype='step', linewidth=2)

        chi2 = np.mean((hhm-hhg)**2/np.fmax(1,hhm))
        ax.text(0.55, 0.875, r"$\chi^2 = $"+f"{chi2:.2f}", transform=ax.transAxes, fontsize=20)

        ax.grid(True, which='both')
        ax.set_title('')
        ax.set_xlabel(xl, fontsize=18); #ax.set_ylabel('count')
        ax.ticklabel_format(axis='y',style='sci', scilimits=(0,0), useMathText=True)
        ax.tick_params(axis='both', which='major', labelsize=14)
        # if 'x' in xl or 'Q' in xl or 't' in xl:
        if log:
            # ax.set_xscale('symlog')
            ax.set_yscale('log')

    axs[-1,-1].set_axis_off()
    axs[-1,-1].text(0.5, 0.6, "MCEG", transform=axs[-1,-1].transAxes, fontsize=75, color='C0', ha='center')
    axs[-1,-1].text(0.5, 0.25, "GAN", transform=axs[-1,-1].transAxes, fontsize=75, color='C1', ha='center')

In [ ]:
for xyr in [[[0.0007, 0.025],[3.5, 60]], [[0.0007, 0.005],[3.5, 20]]]:#, [[0.0007, 0.002],[3.5, 8]]]:
    f, axs = sideBySide2D("Xbj", "Q2", xyr, bins=200, xl=r"$x_B$", yl=r"$Q^2$ [GeV$^2$]", title=None, chi2=False, scilabs='x')
    for ax in axs:
        ax.plot([.1, 0.000805, 0.002015, 0.00293, 0.004522, 0.006, 0.00775, 0.01, 0.0121, 0.015, 0.018, 0.025], [4, 4, 10, 14, 20, 25, 30, 35.5, 40, 45.25, 50, 58.25], 'red', linewidth=3)

In [ ]:
for xyr in [[[3.5, 60],[-0.01, 1.01]], [[3.75, 20],[-0.01, 1.01]]]:#, [[0.0007, 0.002],[3.5, 8]]]:
    f, axs = sideBySide2D("Q2", "t", xyr, bins=200, xl=r"$Q^2$ [GeV$^2$]", yl=r"$-t$ [GeV$^2$]", title=None, chi2=False, scilabs=None)

In [ ]:
xyranges = [[[0.0008, 0.025],[4, 60]], [[0.0008, 0.005],[4, 20]], [[4, 20],[0, 1]], [[4, 25], [0.05, 0.9]]]
xxs = ["Xbj", "Xbj", "Q2", "Q2"]
yys = ["Q2", "Q2", "t", "y"]
xls = [r"$x_B$", r"$x_B$", r"$Q^2$ [GeV$^2$]", r"$Q^2$ [GeV$^2$]"]#r"$\varphi_{HADRON} [deg]$"]
yls = [r"$Q^2$ [GeV$^2$]", r"$Q^2$ [GeV$^2$]", r"$-t$ [GeV$^2$]",  r"$y$"]#r"$-t$ [GeV^$2$]"]
for xx, yy, xl, yl, xyr in zip(xxs, yys, xls, yls, xyranges):
    contourOn2D(xx, yy, xyr, bins=100, levels=15, xl=xl, yl=yl, title=None, chi2=True, scilabs='x' if 'Xbj' in xx else None)